<a href="https://colab.research.google.com/github/Shamitr-Mardikar/LlamaIndex-MovieQuery/blob/main/LlamaIndex_and_RAG_Movie_Query_Model_using_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LlamaIndex and RAG Movie Query model using Gemini API**


Installing all dependencies required.

In [ ]:
!pip install -q llama-index
!pip install -q llama-index-llms-gemini
!pip install -q llama-index-embeddings-gemini
!pip install llama-index-vector-stores-chroma
!pip install chromadb

Importing all the necessary libraries

In [ ]:
import os
import getpass
import chromadb
import re
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings
from llama_index.core import PromptTemplate
from llama_index.embeddings.gemini import GeminiEmbedding

Extracting User's Gemini API Key

Please enter below.

In [ ]:
gemini_api_key = getpass.getpass("Enter your Gemini API Key here : ")

Enter your Gemini API Key here : ··········


Reading the Movie Review txt file using Simple Directory Reader from LlamaIndex.

Remember to add your data to the data folder. If data folder not created, create a data folder and add the document in txt format.

In [ ]:
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
documents

[Document(id_='40c267e8-4d66-4fe6-89fc-3a83a30e8ea6', embedding=None, metadata={'file_path': '/content/data/data.txt', 'file_name': 'data.txt', 'file_type': 'text/plain', 'file_size': 14035, 'creation_date': '2024-04-11', 'last_modified_date': '2024-04-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Movie Review: Iron Man (2008)\r\nCast: Robert Downey Jr. as Tony Stark/Iron Man, Gwyneth Paltrow as Virginia "Pepper" Potts, Jeff Bridges as Obadiah Stane, Terrence Howard as James "Rhodey" Rhodes, Shaun Toub as Yinsen.\r\nPlot: "Iron Man" is a gripping superhero film that marks the beginning of the Marvel Cinematic Universe (MCU) with an electrifying performance from Robert Downey Jr. as Tony Stark, a genius billionaire industrialist who is also a bril

Initialising the Gemini Model

In [ ]:
model_name = "models/embedding-001"
gemini_embedding_model = GeminiEmbedding(model_name = model_name,api_key = gemini_api_key, title="document")

Initialising the LLM.

In [ ]:
llm = Gemini(api_key = gemini_api_key, model_name = "models/gemini-pro")

Creating a new Chroma client and Chroma collection.

In [ ]:
client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = client.get_or_create_collection("quickstart")

Creating a Vector Store for Chroma

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Setting up the LLM and interlinking the Vector Store.

In [ ]:
Settings.llm = llm
Settings.embed_model = gemini_embedding_model
index = VectorStoreIndex.from_documents(documents, storage_context = storage_context)

Loading the chroma client and providing a test query to the LLM engine we just made.

In [ ]:
load_client = chromadb.PersistentClient(path = "./chroma_db")
chroma_collection = load_client.get_collection("quickstart")

vector_store = ChromaVectorStore(chroma_collection = chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store)

test_query_engine = index.as_query_engine()
response = test_query_engine.query("MMLU")
print(response)

The provided context does not mention anything about MMLU, so I cannot answer this question from the provided context.


Using Llama Template to give Gemini a query and initialise it to answer questions relating to the Movie Review Document.

In [ ]:
template = (""" You are an assistant for Question-Answering Tasks. Use the following context to answer questions based off the movie reviews. IF you don't know the answer, just say you don't know. Use 5 sentences max, keep the answer concise.
                Question : {query_str} \nContext: {context_str} \nAnswer: """)

llm_prompt = PromptTemplate(template)

In [ ]:
query_engine = index.as_query_engine(text_qa_template=llm_prompt)

You can ask any Query relating to the Movie Review document including short summaries and Gemini will give it to you!

In [ ]:
question = input()
response = query_engine.query(question)
print(response)

what is the summary of IT
IT is a chilling horror film that follows a group of seven children in the town of Derry, Maine, who are terrorized by a malevolent entity that takes the form of a sinister clown named Pennywise. The children, known as the Losers' Club, must confront their deepest fears and band together to defeat Pennywise, who awakens every 27 years to prey on the town's children.
